In [10]:
#Importing braket libraries
from braket.circuits import Circuit
from braket.aws import AwsDevice

#function to convert a bit string bitstr into a quantum circuit
def bit_string_to_circuit(bitStr):
    circuit=Circuit()
    for i in range(len(bitStr)):
        if bitStr[i]=='1':
            circuit.x(i) #applies pauly x gate flips the state of a qubit from |0⟩ to |1⟩ 
    return circuit

#sample variable for testing
a ='11010'

#prints quantum circuit that prepares corresponding quantum state
print(bit_string_to_circuit(a))



T  : │  0  │
      ┌───┐ 
q0 : ─┤ X ├─
      └───┘ 
      ┌───┐ 
q1 : ─┤ X ├─
      └───┘ 
      ┌───┐ 
q3 : ─┤ X ├─
      └───┘ 
T  : │  0  │


In [11]:
#Importing braket libraries
from braket.circuits import Circuit
from braket.aws import AwsDevice

#function to convert a bit string bitstr into a quantum circuit
def bit_string_to_circuit(bitStr):
    circuit=Circuit()
    for i in range(len(bitStr)):
        if bitStr[i]=='1':
            circuit.x(i) #applies pauly x gate flips the state of a qubit from |0⟩ to |1⟩ 
    return circuit

#sample variable for testing
a =input(int())

#prints quantum circuit that prepares corresponding quantum state
print(bit_string_to_circuit(a))




0 10110


T  : │  0  │
      ┌───┐ 
q0 : ─┤ X ├─
      └───┘ 
      ┌───┐ 
q2 : ─┤ X ├─
      └───┘ 
      ┌───┐ 
q3 : ─┤ X ├─
      └───┘ 
T  : │  0  │


In [12]:
#IMPLEMENTING THE QNN 
# A QNN circuit layout for binary classification. Black horizontal lines represent qubits. 
#Orange and purple boxes represent single- and two-qubit gates respectively.
#The single-qubit gates Rx and Ry and two-qubit gates XX are standard gates defined in Amazon Braket SDK.

In [13]:
# importing standard numpy libraries and optimizers
import numpy as np 
from scipy.optimize import minimize 

#BRAKET IMPORTS
from braket.circuits import Circuit ,Gate, Instruction,circuit, Observable
from braket.aws import AwsDevice,AwsQuantumTask
from braket.devices import LocalSimulator 

#Setting Braket backend to local simulator 
device = LocalSimulator()

# Quantum Neural Net from the QNN figure implemented in Braket
# Inputs: bitStr - data bit string (e.g. '01010101')
#         pars - array of parameters theta (see the QNN figure for more details)

def QNN(bitStr,pars):
    ##size of QNN circuit
    nQbts = len(bitStr)+1   #extra qubit allocated for the label 
    
    ##intializing the circuit
    qnn = Circuit()
    

## add single-qubit X rotation to the label qubit, 
    ## initialize the input state to the one specified by bitStr
    ## add single-qubit Y rotations to data qubits, 
    ## add XX gate between qubit i and the label qubit, 
    
    qnn.rx(nQbts-1, pars[0])
    for i in range(nQbts-1):
        angles=pars[2*i+1:2*i+1+2]
        if bitStr[i] =='1': # by default Braket sets input states to '0', 
                            # qnn.x(ind) flips qubit number ind to state |1\ 
            qnn.x(i)
        qnn.ry(i,angles[0]).xx(i,nQbts-1,angles[1])
        
               
    ## add z observable to label qubit
    observZ=Observable.Z()
    qnn.expectation(observZ,target=[nQbts-1])
               
    return qnn
        
    
    

In [14]:
#Function to compute the label of a given feature of a bit string

def parity(bitStr):
    return bitStr.count('1')%2
## Log loss function L(theta,phi) for a given training set trainSet
## inputs: trainSet - array of feature bit strings e.g. ['0101','1110','0000'] 
##         pars - quantum neural net parameters theta (See the QNN figure)
##         device -  Braket backend that will compute the log loss

def loss(trainSet,pars,device):
    loss=0.0
    for i in range(np.size(trainSet)):
        #to run QNN on braket device
        task=device.run(QNN(trainSet[i],pars),shots=0)
        ##to retrieve the run results<Z>
        result = task.result()
        
        if parity(trainSet[i])==0:
            loss+= -np.log2(1.0-0.5*(1.0-result.values[0]))
        else:
            loss+=-np.log2(0.5*(1.0-result.values[0]))
    print("Current value of the loss function: ", loss)
    return loss
            


###### Training the QNN using gradient-based optimizer 
nBits = 10 # number of bits per data string 

#Random training set consisting all of 11 10-bit data strings
trainSet = ['1101011010',
            '1000110011',
            '0101001001',
            '0010000110',
            '0101111010',
            '0000100010',
            '1001010000',
            '1100110001',
            '1000010001',
            '0000111101',
            '0000000001']
##Initial assignment of QNN parameters theta and phi ( random angles in [-pi,pi])
parsQ=2*np.pi*np.random.rand(2*nBits+1)-np.pi

## Run minimization
res= minimize(lambda pars: loss(trainSet, pars, device),parsQ,method='BFGS',options={'disp' :True})


In [20]:
nBits = 10 # number of bits per data string 

#Random training set consisting all of 11 10-bit data strings
trainSet = ['1101011010',
            '1000110011',
            '0101001001',
            '0010000110',
            '0101111010',
            '0000100010',
            '1001010000',
            '1100110001',
            '1000010001',
            '0000111101',
            '0000000001']
##Initial assignment of QNN parameters theta and phi ( random angles in [-pi,pi])
parsQ=2*np.pi*np.random.rand(2*nBits+1)-np.pi

## Run minimization
res= minimize(lambda pars: loss(trainSet, pars, device),parsQ,method='BFGS',options={'disp' :True})

Current value of the loss function:  11.003518197276579
Current value of the loss function:  11.003518197299611
Current value of the loss function:  11.00351819727584
Current value of the loss function:  11.003518197285164
Current value of the loss function:  11.0035181973303
Current value of the loss function:  11.003518197503103
Current value of the loss function:  11.00351819726989
Current value of the loss function:  11.00351819721747
Current value of the loss function:  11.003518197304553
Current value of the loss function:  11.003518197249365
Current value of the loss function:  11.003518197279416
Current value of the loss function:  11.003518197232106
Current value of the loss function:  11.003518197262512
Current value of the loss function:  11.00351819731136
Current value of the loss function:  11.003518197226684
Current value of the loss function:  11.003518197328775
Current value of the loss function:  11.003518196785308
Current value of the loss function:  11.00351819822342

In [21]:
for ind in range(2**nBits):
    data = format(ind, '0'+str(nBits)+'b')
    task = device.run(QNN(data, res.x), shots=100)
    result = task.result()
    if (data in trainSet):
        inSet = 'in the training set'
    else:
        inSet = 'NOT in the training set'
    print('Feature:', data, '| QNN predicted parity: ', 0.5*(1-result.values[0]), ' | ', inSet) 
    print('---------------------------------------------------')
        

Feature: 0000000000 | QNN predicted parity:  0.0  |  NOT in the training set
---------------------------------------------------
Feature: 0000000001 | QNN predicted parity:  1.0  |  in the training set
---------------------------------------------------
Feature: 0000000010 | QNN predicted parity:  1.0  |  NOT in the training set
---------------------------------------------------
Feature: 0000000011 | QNN predicted parity:  0.0  |  NOT in the training set
---------------------------------------------------
Feature: 0000000100 | QNN predicted parity:  1.0  |  NOT in the training set
---------------------------------------------------
Feature: 0000000101 | QNN predicted parity:  0.0  |  NOT in the training set
---------------------------------------------------
Feature: 0000000110 | QNN predicted parity:  0.0  |  NOT in the training set
---------------------------------------------------
Feature: 0000000111 | QNN predicted parity:  1.0  |  NOT in the training set
------------------------

In [24]:
correct_predictions = 0
total_inputs = 2 ** nBits

for ind in range(total_inputs):
    data = format(ind, '0'+str(nBits)+'b')
    task = device.run(QNN(data, res.x), shots=100)
    result = task.result()
    predicted_parity = 0.5 * (1 - result.values[0])

    # Assuming you have a function to compute actual parity based on input data
    actual_parity = compute_actual_parity(data)

    if predicted_parity == actual_parity:
        correct_predictions += 1

    if data in trainSet:
        inSet = 'in the training set'
    else:
        inSet = 'NOT in the training set'

    print('Feature:', data, '| QNN predicted parity:', predicted_parity, '|', inSet)
    print('---------------------------------------------------')

accuracy = (correct_predictions / total_inputs) * 100
print('Accuracy:', accuracy, '%')

NameError: name 'compute_actual_parity' is not defined

NameError: name 'y_pred' is not defined